In [8]:
import pandas as pd
import sqlite3
from pathlib import Path

orders_df = pd.read_csv("orders.csv")
users_df = pd.read_json("users.json")

conn = sqlite3.connect(":memory:")
sql_text = Path("restaurants.sql").read_text(encoding="utf-8")
conn.executescript(sql_text)
restaurants_df = pd.read_sql("SELECT * FROM restaurants", conn)

merged_df = orders_df.merge(
    users_df,
    on="user_id",
    how="left"
)

merged_df = merged_df.merge(
    restaurants_df,
    on="restaurant_id",
    how="left"
)

merged_df["order_date"] = pd.to_datetime(
    merged_df["order_date"],
    errors="coerce"
)

merged_df["quarter"] = merged_df["order_date"].dt.to_period("Q")

merged_df.to_csv("final_food_delivery_dataset.csv", index=False)

# Q1: Gold revenue by city
print(
    merged_df[merged_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

# Q2: Average order value by cuisine
print(
    merged_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

# Q3: Users with total spend > 1000
print(
    (merged_df.groupby("user_id")["total_amount"].sum() > 1000).sum()
)

# Q4: Revenue by rating range
merged_df["rating_range"] = pd.cut(
    merged_df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"],
    include_lowest=True
)

print(
    merged_df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

# Q5: Gold average order value by city
print(
    merged_df[merged_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

# Q6: Restaurant count vs revenue by cuisine
print(
    merged_df.groupby("cuisine").agg(
        restaurant_count=("restaurant_id", "nunique"),
        total_amount=("total_amount", "sum")
    ).sort_values("restaurant_count")
)

# Q7: Percentage of orders by Gold members
print(
    round(
        merged_df[merged_df["membership"] == "Gold"].shape[0]
        / merged_df.shape[0] * 100
    )
)

# Q8: Highest AOV restaurant (< 20 orders)
# Q8: Highest AOV restaurant (< 20 orders)
print(
    merged_df.groupby("restaurant_name_y")
    .agg(
        order_count=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
    .query("order_count < 20")
    .sort_values("avg_order_value", ascending=False)
    .head(1)
)



# Q9: Revenue by membership and cuisine
print(
    merged_df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

# Q10: Revenue by quarter
print(
    merged_df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

# Total orders by Gold members
print(merged_df[merged_df["membership"] == "Gold"].shape[0])

# Total revenue from Hyderabad
print(round(
    merged_df[merged_df["city"] == "Hyderabad"]["total_amount"].sum()
))

# Distinct users
print(merged_df["user_id"].nunique())

# Average order value for Gold members
print(round(
    merged_df[merged_df["membership"] == "Gold"]["total_amount"].mean(), 2
))

# Orders with rating ≥ 4.5
print(merged_df[merged_df["rating"] >= 4.5].shape[0])

# Gold orders in Chennai
print(
    merged_df[
        (merged_df["membership"] == "Gold") &
        (merged_df["city"] == "Chennai")
    ].shape[0]
)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64
2544
rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64
city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64
         restaurant_count  total_amount
cuisine                                
Chinese               120    1930504.65
Indian                126    1971412.58
Italian               126    2024203.80
Mexican               128    2085503.09
50
                   order_count  avg_order_value
restaurant_name_y                              
Restaurant_294              13      1040.222308
membership  cuisine
Regular     Mexican    1072943.30
            Ita

C:\Users\veera\AppData\Local\Temp\ipykernel_18764\3830359177.py:25: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  merged_df["order_date"] = pd.to_datetime(
C:\Users\veera\AppData\Local\Temp\ipykernel_18764\3830359177.py:64: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df.groupby("rating_range")["total_amount"]
